## Importing Essential Libraries

In [14]:
from google.adk.agents import LlmAgent , Agent , SequentialAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory, preload_memory , google_search ,AgentTool
from google.genai import types
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import List, Optional

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory, preload_memory
from google.genai import types
print("✅ ADK components imported successfully.")


✅ ADK components imported successfully.


## Setup Gemini Api Key

In [15]:
try:
    GOOGLE_API_KEY = load_dotenv("GOOGLE_API_KEY")
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


## Configure Retry Options

In [16]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

## Creating the Agent Instance

In [ ]:
#!adk create patient_intake_agent --model gemini-2.0-flash --api_key $GOOGLE_API_KEY

## Initialize Memory

In [ ]:

google_search_agent = LlmAgent(
    name="GoogleSearchAgent",
    model=Gemini(
        model_name="gemini-2.0-flash",
        retry_options=retry_config,
    ),
    description='''
You are GoogleSearchAgent, an expert in finding accurate and relevant information using Google Search.
Your task is to assist the main agent by performing Google searches based on user queries and returning concise summaries of the search results. ''',
    instruction = '''
You will be provided with a user's query. Use Google Search to find the most relevant and up-to-date information. It should be fact based and objective.
Store the search results in 'google_search_output'.
''',
    tools=[google_search],
    output_key="google_search_output"
)

In [ ]:
test_agent = LlmAgent(
    name="test_agent",
    model=Gemini(
        model_name="gemini-2.0-flash",
        retry_options=retry_config,
    ),
    description = 'just greet the user',
    instruction = 'jsut greet the user with good morning',
    tools = [load_memory],
    output_key="test_result",
    sub_agents = [google_search_agent]
)

In [ ]:
combine_agent = Agent(
    name="combine_agent",
    model=Gemini(
        model_name="gemini-2.0-flash",
        retry_options=retry_config,
    ),
    description = 'combine the results from sub agents',
    instruction = 'combine the results from sub agents into a single output',
    tools = [preload_memory],
    output_key="final_output",
    sub_agents = [test_agent, google_search_agent]
)

In [ ]:
root_agent = combine_agent